In [ ]:
from voxrow.web.domain.value_objects import Settings

In [ ]:
settings: Settings = Settings()

# Duckdb

In [ ]:
%load_ext magic_duckdb
%dql -t relation
%dql SELECT 'Hanya Tes' ini;

In [ ]:
from os import environ

In [ ]:
environ["AWS_ACCESS_KEY_ID"] = settings.cloudflare_r2.aws_access_key_id.get_secret_value()
environ["AWS_SECRET_ACCESS_KEY"] = settings.cloudflare_r2.aws_secret_access_key.get_secret_value()

In [ ]:
%%dql -j
CREATE OR REPLACE
SECRET (
    TYPE r2,
    PROVIDER credential_chain,
    CHAIN env,
    ENDPOINT '{{ settings.cloudflare_r2.endpoint_url.host }}'
);

# API

In [ ]:
from fastapi.testclient import TestClient

from voxrow.web.entrypoint import app

In [ ]:
client: TestClient = TestClient(app)

## /idx/stock-summary

In [ ]:
%%dql
CREATE OR REPLACE
TEMP TABLE
    idx_stock_summary
AS
SELECT
    "filename"
FROM
    READ_BLOB('r2://datalake/idx.co.id/GetStockSummary/*.json.gz')
;

In [ ]:
%%dql
SELECT
    "date"."year" || '-' || "date"."month" AS "month",
    COUNT(*) AS files
FROM
    idx_stock_summary,
    LATERAL (
        SELECT
            REGEXP_EXTRACT(
                "filename",
                '.*/(\d{4}).(\d{2}).\d{2}.json.gz',
                ['year', 'month']
            ) AS "date"
    )
GROUP BY
    1
ORDER BY
    1
;

In [ ]:
import random
from calendar import monthrange
from datetime import date
from time import sleep

from httpx import Response

In [ ]:
# Oldest date is 2020-01-02
year: int = 2024
month: int = 12
_, days = monthrange(year, month)

for day in range(1, days + 1):
    current_date: date = date(year, month, day)

    if current_date.strftime("%a") not in ("Sat", "Sun"):
        resp: Response = client.get(
            "/idx/stock-summary",
            headers=dict(Authorization=f"Bearer {settings.cron_secret.get_secret_value()}"),
            params=dict(date=str(current_date)),
            timeout=60 * 1.5,
        )

        print(f"{current_date} | status_code: {resp.status_code}")
        sleep(random.uniform(3.0, 5.0))